## opt

In [27]:
import torch
import os
from os.path import join
import numpy as np
import pickle
from tqdm import *
import trimesh
from manotorch.manolayer import ManoLayer
from pysdf import SDF

In [43]:
rootpath = '/root/code/seqs/0303_data'
obj_path = '/root/code/seqs/object'
contactgen_path = 'final_results/contactgen_500_2000/'
# obj_pose_path ''
# path = join('save/0303_stage0_1obj/samples_0303_stage0_1obj_000030000_seed10_predefined/pred_obj',f'{seq}_pred_obj_and_goal.npy')

## 把所有的hand trans转为需要减去joint0的格式

align_hand_param.npy

In [ ]:
seqs = sorted(os.listdir(contactgen_path))
manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano', side='right')
hand_faces = manolayer.th_faces

# joints = output.joints  + hand_trans.unsqueeze(1)
# hand_verts = output.verts  + hand_trans.unsqueeze(1)
# joints = output.joints - output.joints[:, 0].unsqueeze(1) + hand_trans.unsqueeze(1)
for seq in seqs:
    res_path = join(contactgen_path,seq,'results')
    ids = sorted(os.listdir(res_path))
    for id in ids:
        idpath = join(res_path,id)
        old_hand_params = torch.tensor(np.load(join(idpath,'new_hand_params.npy')))
        print(old_hand_params.shape)
        hand_trans = old_hand_params[:3].unsqueeze(0)
        hand_theta = old_hand_params[3:51].unsqueeze(0)
        mano_beta = old_hand_params[51:].unsqueeze(0)
        output = manolayer(hand_theta, mano_beta)
        joint_offset = output.joints[:, 0].unsqueeze(1)
        new_hand_trans = hand_trans + joint_offset
        align_hand_params = old_hand_params.squeeze(0)
        align_hand_params[:3] = new_hand_trans.squeeze(0)
        align_hand_params = align_hand_params.numpy()
        np.save(join(idpath,'align_hand_params.npy'), align_hand_params)
    #     break
    # break

# 计算优化需要的hand T和obj_tip参数 之前的

In [ ]:
# datapath = '/root/code/seqs/gazehoi_list_test_0303.txt'
# with open(datapath,'r') as f:
#     info_list = f.readlines()
# seqs = []
# for info in info_list:
#     seq = info.strip()
#     seqs.append(seq)

data_dict = {}
stage1_res = np.load('save/8000_DSG/samples_final_000008000_seed10_new_dsg750_gaze/results.npy',allow_pickle=True).item()
print(stage1_res.keys())
hand_motions = torch.tensor(stage1_res['motion']).float()
seqs = stage1_res['seqs']
hint = torch.tensor(stage1_res['hint']).float()

for i in range(len(seqs)):
    seq = seqs[i]
    offset = torch.tensor(np.load(join(contactgen_path,seq,'offset.npy')))
    seq_path = join(rootpath,seq)
    meta_path = join(seq_path,'meta.pkl')
    with open(meta_path, 'rb')as f:
        meta = pickle.load(f)
    gaze_obj = meta['gaze_obj']
    gt_hand_params = np.load(join(seq_path,'mano/poses_right.npy'))
    mano_beta = torch.tensor(gt_hand_params[0,51:])
    obj_mesh = trimesh.load(join(obj_path,gaze_obj,'simplified_scan_processed.obj'))
    obj_verts = torch.tensor(obj_mesh.vertices).float()
    obj_faces = obj_mesh.faces

    stage0_res = np.load(join('save/0303_stage0_1obj/samples_0303_stage0_1obj_000030000_seed10_predefined/pred_obj',f'{seq}_pred_obj_and_goal.npy'),allow_pickle=True).item()
    # print(stage0_res.keys())
    obj_pose = torch.tensor(stage0_res['pred_obj_pose']).float()
    goal_index = stage0_res['goal_index']
    if goal_index<=59:
        goal_hand_pose = hint[i,goal_index]
        
    else:
        goal_hand_pose = hint[i,-1]
        # print(seq)
    # print(seq,goal_hand_pose)
    # goal_hand_pose[:3] = goal_hand_pose[:3] + offset
    stage2_obj_pose = obj_pose[goal_index:]
    goal_obj_pose = obj_pose[goal_index]

    trans_obj_verts = get_trans_obj_verts(obj_verts,obj_pose)
    goal_obj_verts = trans_obj_verts[0]

    stage2_hand_T = get_hand_T(stage2_obj_pose, trans_obj_verts[goal_index:], goal_hand_pose)
    np.save(join(seq_path,'stage2_hand_T.npy'),stage2_hand_T.numpy())
    tip_obj_ids, tip_obj_dis,hand_verts,hand_faces = get_tip_obj(goal_obj_pose, goal_obj_verts, goal_hand_pose, mano_beta)
    # print(stage2_hand_T.shape)
    # tip_obj = torch.cat(tip_obj_ids,tip_obj_dis)
    # print(tip_obj.shape)
    np.save(join(seq_path,'tip_obj_ids.npy'), tip_obj_ids.numpy())
    np.save(join(seq_path,'tip_obj_dis.npy'), tip_obj_dis.numpy())
    
    # print(np.load(join(seq_path,'tips_closest_id_and_dis.npy')))
    
    # render = {'hand_verts':hand_verts.squeeze(0).numpy(),'hand_faces':hand_faces,'obj_verts':goal_obj_verts.numpy(),'obj_faces':obj_faces}
    # np.save(join(f'final_results/check_opt_offset/{seq}_for_render.npy'),render)
    # break
    print("#################")

    

In [62]:
def get_tip_obj(goal_obj_pose, obj_verts, goal_hand_pose,mano_beta):
    manolayer = ManoLayer(mano_assets_root='/root/code/CAMS/data/mano_assets/mano', side='right')
    hand_faces = manolayer.th_faces
    hand_trans = goal_hand_pose[:3].unsqueeze(0)
    hand_theta = goal_hand_pose[3:51].unsqueeze(0)
    mano_beta = mano_beta.unsqueeze(0)
    output = manolayer(hand_theta, mano_beta)
    # joints = output.joints  + hand_trans.unsqueeze(1)
    # hand_verts = output.verts  + hand_trans.unsqueeze(1)
    joints = output.joints - output.joints[:, 0].unsqueeze(1) + hand_trans.unsqueeze(1)
    hand_verts = output.verts - output.joints[:, 0].unsqueeze(1) + hand_trans.unsqueeze(1)
    print(hand_verts.shape)
    tips = [15,3,6,12,9]
    tips_T = joints[:,tips].squeeze(0)
    print(tips_T.shape)
    dis = torch.cdist(tips_T,obj_verts)
    tip_obj_dis,tip_obj_ids = dis.min(dim=-1)
    print(tip_obj_ids)
    print(tip_obj_dis)


    
    return tip_obj_ids, tip_obj_dis,hand_verts,hand_faces
    
def get_hand_T(stage2_obj_pose, trans_obj_verts, goal_hand_pose):
    stage2_len = stage2_obj_pose.shape[0]
    update_hand_T = torch.zeros((stage2_len,3))
    update_hand_T[0] = goal_hand_pose[:3]


    obj_R = stage2_obj_pose[:,:3,:3]
    obj_T = stage2_obj_pose[:,:3,3]
    reference_obj_rot = obj_R[0]

    # trans_obj_verts = get_trans_obj_verts(obj_verts,stage2_obj_pose)
    goal_obj_verts = trans_obj_verts[0]
    goal_hand_trans = goal_hand_pose[:3].unsqueeze(0)
    dis = torch.cdist(goal_hand_trans,goal_obj_verts)
    min_dis,obj_idx = dis.min(dim=-1)
    contact_normal = goal_hand_trans - goal_obj_verts[obj_idx]
    
    for i in range(1, stage2_len):
        relative_rot_mat = torch.matmul(obj_R[i], reference_obj_rot.inverse())
        curr_contact_normal = torch.matmul(relative_rot_mat, contact_normal.T).squeeze(-1)
        update_hand_T[i] = trans_obj_verts[i, obj_idx] + curr_contact_normal
    return update_hand_T

def get_trans_obj_verts(obj_verts,obj_pose):
    """
    obj_verts: (N,3) -- T,N
    obj_pose: (T,3,4)
    """
    nf = obj_pose.shape[0]
    N = obj_verts.shape[0]
    # obj_pose = obj_pose.unsqueeze(1).repeat(1,N,1,1)
    obj_R = obj_pose[:,:3,:3]
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    obj_T = obj_pose[:,:3,3].unsqueeze(1)
    
    obj_verts = obj_verts.unsqueeze(0).repeat(nf,1,1)
    trans_obj_verts = torch.einsum('fpn,fnk->fpk',obj_verts,obj_R) + obj_T
    return trans_obj_verts

## 修正后 获得优化需要的hand T和tip obj，for video visualization

In [82]:
# datapath = '/root/code/seqs/gazehoi_list_test_0303.txt'
# with open(datapath,'r') as f:
#     info_list = f.readlines()
# seqs = []
# for info in info_list:
#     seq = info.strip()
#     seqs.append(seq)

data_dict = {}
stage1_res = np.load('save/0303_stage1_simple/samples_0303_stage1_simple_000008000_seed10_0312_val_gaze_align/results.npy',allow_pickle=True).item()
print(stage1_res.keys())
hand_motions = torch.tensor(stage1_res['motion']).float()
print(hand_motions.shape)
seqs = stage1_res['seqs']
hint = torch.tensor(stage1_res['hint']).float()
select_seqs = ['1354']
goal_index = 35
for i in range(len(seqs)):
    seq = seqs[i]
    if seq not in select_seqs:
        continue
    offset = torch.tensor(np.load(join(contactgen_path,seq,'offset.npy')))
    seq_path = join(rootpath,seq)
    meta_path = join(seq_path,'meta.pkl')
    with open(meta_path, 'rb')as f:
        meta = pickle.load(f)
    gaze_obj = meta['gaze_obj']
    gt_hand_params = np.load(join(seq_path,'mano/poses_right.npy'))
    mano_beta = torch.tensor(gt_hand_params[0,51:])
    obj_mesh = trimesh.load(join(obj_path,gaze_obj,'simplified_scan_processed.obj'))
    obj_verts = torch.tensor(obj_mesh.vertices).float()
    obj_faces = obj_mesh.faces

    stage0_res = np.load(join('save/0303_stage0_1obj/samples_0303_stage0_1obj_000030000_seed10_predefined/pred_obj',f'{seq}_pred_obj_and_goal.npy'),allow_pickle=True).item()
    obj_pose = torch.tensor(stage0_res['pred_obj_pose']).float()
    
    goal_hand_pose = hand_motions[i,goal_index]
    stage2_obj_pose = obj_pose[goal_index:]
    goal_obj_pose = obj_pose[goal_index]

    trans_obj_verts = get_trans_obj_verts(obj_verts,obj_pose)
    goal_obj_verts = trans_obj_verts[goal_index]

    stage2_hand_T = get_hand_T(stage2_obj_pose, trans_obj_verts[goal_index:], goal_hand_pose)
    np.save(join(seq_path,'vis_stage2_hand_T.npy'),stage2_hand_T.numpy())
    tip_obj_ids, tip_obj_dis,hand_verts,hand_faces = get_tip_obj(goal_obj_pose, goal_obj_verts, goal_hand_pose, mano_beta)
    # print(stage2_hand_T.shape)
    # tip_obj = torch.cat(tip_obj_ids,tip_obj_dis)
    # print(tip_obj.shape)
    np.save(join(seq_path,'vis_tip_obj_ids.npy'), tip_obj_ids.numpy())
    np.save(join(seq_path,'vis_tip_obj_dis.npy'), tip_obj_dis.numpy())
    
    # print(np.load(join(seq_path,'tips_closest_id_and_dis.npy')))
    
    render = {'hand_verts':hand_verts.squeeze(0).numpy(),'hand_faces':hand_faces,'obj_verts':goal_obj_verts.numpy(),'obj_faces':obj_faces}
    new_path = f'final_results/check_vis/{seq}'
    os.makedirs(new_path,exist_ok=True)
    np.save(join(f'final_results/check_vis/{seq}/{seq}_for_render.npy'),render)
    # break
    print("#################")

    

dict_keys(['motion', 'lengths', 'hint', 'seqs', 'num_samples', 'num_repetitions'])
torch.Size([130, 60, 51])
torch.Size([1, 778, 3])
torch.Size([5, 3])
tensor([ 7985, 11360,  4571, 10392,  2273])
tensor([0.0387, 0.0335, 0.0367, 0.0200, 0.0612])
#################
